<a href="https://colab.research.google.com/github/vvvu/potential-chainsaw/blob/main/pytorch-tutorial/%5BAdvanced%5D_Generative_Adversarial_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# Hyper Parameters
latent_size = 64
hidden_size = 256
image_size = 784
num_epochs = 200
batch_size = 100
sample_dir = 'samples'

# Create a directory if not exists
if not os.path.exists(sample_dir):
  os.makedirs(sample_dir)

In [4]:
# Image Preprocessing
transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean = [0.5], # 1 for greyscale channels
                                 std = [0.5])
])

In [5]:
# MNIST dataset
mnist = torchvision.datasets.MNIST(root='./data/',
                                   train=True,
                                   transform=transform,
                                   download=True)

# Data loader
data_loader = torch.utils.data.DataLoader(dataset=mnist,
                                          batch_size=batch_size, 
                                          shuffle=True)

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [6]:
# Discriminator
'''
784 -> 256 -> 256 -> 1
'''
D = nn.Sequential(
    nn.Linear(image_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, 1),
    nn.Sigmoid()
)

# Generator
'''
64 -> 256 -> 256 -> 784
'''
G = nn.Sequential(
    nn.Linear(latent_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, image_size),
    nn.Tanh()
)

D = D.to(device)
G = G.to(device)

In [7]:
# Loss & Optimizer
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr = 0.0002)
g_optimizer = torch.optim.Adam(G.parameters(), lr = 0.0002)

def denorm(x):
  out = (x + 1) / 2
  '''
  torch.clamp
  - 将输入Input Tensor的每个元素夹紧到区间[min, max]之间，并返回结果得到一个New Tensor
  y = {
    min, if x < min
    x, if min <= x <= max
    max, if x > max
  }
  - 事实上实现它的实现为y = min(max(x, min_value),max_value)
  '''
  return out.clamp(0, 1)

def reset_grad():
  d_optimizer.zero_grad()
  g_optimizer.zero_grad()

# Start training
total_step = len(data_loader)
for epoch in range(num_epochs):
  for i, (images, _) in enumerate(data_loader):
    images = images.reshape(batch_size, -1).to(device)

    # Create the labels which are later used as input for the BCE loss
    real_labels = torch.ones(batch_size, 1).to(device)
    fake_labels = torch.zeros(batch_size, 1).to(device)

    '''
    Train the discriminator
    '''

    # Compute BCE_Loss using real images where BCE_Loss(x, y): 
    # - y * log(D(x)) - (1 - y) * log(1 - D(x))

    # Second term of the loss is always zero since real_labels == 1
    outputs = D(images)
    d_loss_real = criterion(outputs, real_labels)
    real_score = outputs

    # Compute BCELoss using fake images
    # First term of the loss is always zero since fake_labels == 0
    z = torch.randn(batch_size, latent_size).to(device)
    '''
    随机生成一个初始数据，放入G()中，我们期望G(z)生成的数据可以尽可能的骗过Discriminator
    '''
    fake_images = G(z)
    outputs = D(fake_images)
    d_loss_fake = criterion(outputs, fake_labels)
    fake_score = outputs

    # Backprop and optimize
    d_loss = d_loss_real + d_loss_fake
    reset_grad()
    d_loss.backward()
    d_optimizer.step()

    '''
    Train the generator
    '''

    # Compute loss with fake images
    z = torch.randn(batch_size, latent_size).to(device)
    fake_images = G(z)
    outputs = D(fake_images)

    # We train G to maximize log(D(G(z))) instend of minimizing log(1-D(G(z)))
    # For the reason, see the last paragraph of section 3. https://arxiv.org/pdf/1406.2661.pdf
    g_loss = criterion(outputs, real_labels)

    # Backprop and optimize
    reset_grad()
    g_loss.backward()
    g_optimizer.step()

    if (i + 1) % 200 == 0:
      print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}'
            .format(epoch, num_epochs, i + 1, total_step, d_loss.item(), g_loss.item(),
                    real_score.mean().item(), fake_score.mean().item()))
      
  '''
  Save real images
  '''
  if (epoch + 1) == 1:
    images = images.reshape(images.size(0), 1, 28, 28)
    save_image(denorm(images), os.path.join(sample_dir, 'real_images.png'))

  '''
  Save sampled images
  '''
  fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
  save_image(denorm(fake_images), os.path.join(sample_dir, 'fake_images-{}.png'.format(epoch + 1)))

Epoch [0/200], Step [200/600], d_loss: 0.0411, g_loss: 4.0127, D(x): 1.00, D(G(z)): 0.04
Epoch [0/200], Step [400/600], d_loss: 0.0449, g_loss: 5.9045, D(x): 0.98, D(G(z)): 0.03
Epoch [0/200], Step [600/600], d_loss: 0.0722, g_loss: 5.6134, D(x): 0.97, D(G(z)): 0.03
Epoch [1/200], Step [200/600], d_loss: 0.0595, g_loss: 5.0628, D(x): 0.98, D(G(z)): 0.02
Epoch [1/200], Step [400/600], d_loss: 0.2649, g_loss: 5.0675, D(x): 0.94, D(G(z)): 0.11
Epoch [1/200], Step [600/600], d_loss: 0.0384, g_loss: 4.8486, D(x): 0.99, D(G(z)): 0.03
Epoch [2/200], Step [200/600], d_loss: 0.3657, g_loss: 4.8178, D(x): 0.84, D(G(z)): 0.11
Epoch [2/200], Step [400/600], d_loss: 0.1522, g_loss: 4.8840, D(x): 0.95, D(G(z)): 0.05
Epoch [2/200], Step [600/600], d_loss: 0.0732, g_loss: 4.6780, D(x): 0.98, D(G(z)): 0.04
Epoch [3/200], Step [200/600], d_loss: 0.2469, g_loss: 3.8014, D(x): 0.94, D(G(z)): 0.10
Epoch [3/200], Step [400/600], d_loss: 0.2649, g_loss: 3.2238, D(x): 0.88, D(G(z)): 0.06
Epoch [3/200], Step [

In [8]:
# Save the model checkpoints
torch.save(G.state_dict(), 'G.ckpt')
torch.save(D.state_dict(), 'D.ckpt')